In [ ]:
#| default_exp read_data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.test import *
from sys import platform
import numpy as np
from pathlib import Path
from typing import Tuple

from cupid_matching.matching_utils import Matching

In [ ]:
#| export

def get_root_dir(
) -> Path:
    """ returns the package directory """
    root_dir = Path.cwd().parent
    if platform in ["linux", "linux2"]:        # we are deploying
        root_dir = Path.cwd().parent
    return root_dir

In [ ]:
#| export

def read_margins(
    data_dir: Path      # the data directory
    ) -> Tuple[np.ndarray, np.ndarray]: 
    """reads and returns the margins for men and for women """
    nx = np.loadtxt(data_dir / "nx70n.txt")
    my = np.loadtxt(data_dir / "my70n.txt")
    return nx, my

def read_marriages(
    data_dir: Path      # the data directory
    ) -> Tuple[np.ndarray, np.ndarray]: 
    """ reads and returns the marriages and the variances"""
    muxy = np.loadtxt(data_dir / "muxy70nN.txt")
    varmus = np.loadtxt(data_dir / "varmus70nN.txt")
    return muxy, varmus

In [ ]:
#| export 

def reshape_varcov(
    varmus: np.ndarray,      #  muxy row major, then  mux0, then mu0y packed in both dimensions
    ncat_men: int,           # the number of types of men
    ncat_women: int,         # the number of types of women
    n_households_pop: int    #  the number of households in the population
) -> tuple[np.ndarray]:      #  the 6 constituent blocks of the normalized variance-covariance
    """ splits the variance-covariance matrix 
    and divides it by the square of the total number of households in the population
    """
    n_prod_categories = ncat_men * ncat_women
    varmus_xyzt = varmus[:n_prod_categories, :n_prod_categories]
    varmus_xyz0 = varmus[:n_prod_categories,
                                n_prod_categories:(n_prod_categories + ncat_men)]
    varmus_xy0t = varmus[:n_prod_categories,
                                (n_prod_categories + ncat_men):]
    varmus_x0z0 = varmus[n_prod_categories:(n_prod_categories + ncat_men),
                                n_prod_categories:(n_prod_categories + ncat_men)]
    varmus_x00y = varmus[n_prod_categories:(n_prod_categories + ncat_men), 
                         (n_prod_categories + ncat_men):]
    varmus_0y0t = varmus[(n_prod_categories + ncat_men):, 
                         (n_prod_categories + ncat_men):]
    varcovs = (varmus_xyzt, varmus_xyz0, varmus_xy0t,
              varmus_x0z0, varmus_x00y, varmus_0y0t)
    N2 = n_households_pop * n_households_pop
    varcovs = tuple(v/N2 for v in varcovs)
    return varcovs


def normalize_mus(
    mus: Matching,         # muxy, mux0, mu0y
    n_households_pop: int    # the number of households in the population
) -> Tuple[Matching, np.ndarray, np.ndarray]:  # the normalized (muxy, mux0, mu0y), nx, my
    """ normalizes the marriages and margins by dividing by the total number of households in the population"""
    muxy, *_, nx, my = mus.unpack()
    muxy_norm = muxy / n_households_pop
    nx_norm = nx / n_households_pop
    my_norm = my / n_households_pop
    mus_norm = Matching(muxy_norm, nx_norm, my_norm)
    return mus_norm



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()